In [3]:
!pip install bokeh
!pip install bkcharts
!pip install pandas
!pip install numpy
!pip install geopy
!pip install statistics
!pip install tqdm
!pip install folium

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import folium
from folium.plugins import HeatMap
from bokeh.charts import Histogram, Bar, Donut, output_notebook, show, defaults
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, ranges, LabelSet

/home/nalbertg/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/nalbertg/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/nalbertg/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


# Carregando base de dados

In [5]:
farmacias = pd.read_csv('datasets/mediPy/farmacia_popular_estabelecimentonone.csv')

## As 5 primeras unidades

In [6]:
farmacias.head()

,lat,long,nu_ddd_farmacia,nu_telefone_farmacia,nu_cep_farmacia,no_bairro_farmacia,ds_endereco_farmacia,no_farmacia,no_cidade,uf
0,-32.236851,-53.088458,54,33131900,96330000,CENTRO,"AVENIDA VISCONDE DE MAUA, 383",COMERCIO DE MEDICAMENTOS BRAIR LTDA,ARROIO GRANDE,RS
1,-31.753500,-52.312792,54,33350100,96085470,AREAL,AVENIDA DOMINGOS DE ALMEIDA 825 LOJA 01,COMERCIO DE MEDICAMENTOS BRAIR LTDA,PELOTAS,RS
2,-23.084332,-47.215462,19,38345604,13330060,CENTRO,"RUA VINTE E QUATRO DE MAIO, 1036",DROGAN DROGARIAS LTDA,INDAIATUBA,SP
3,-27.081033,-53.027768,49,33254232,89885000,CENTRO,"AVENIDA SANTA CATARINA, Nº 101 - A",DROGARIA PATRICIA LTDA,SAO CARLOS,SC
4,-12.993305,-38.457390,11,32747400,41810315,PITUBA,RUA ARISTIDES FRAGA LIMA 51,DROGARIA SAO PAULO S.A.,SALVADOR,BA


## Removendo elementos irrelevantes

In [7]:
del farmacias['nu_ddd_farmacia']
del farmacias['nu_telefone_farmacia']
del farmacias['no_bairro_farmacia']
del farmacias['ds_endereco_farmacia']
del farmacias['no_cidade']

## Ordenação das regiões

In [9]:
farmacias.sort_values('uf',inplace = True)

## Quantitativo farmacias/UF

In [11]:
def quantidade_uf():
    quantidades_ufs = {}
    for index in range(0, len(farmacias)):
            uf = farmacias.loc[index]['uf']
            if len(str(uf))==2:
                if uf not in quantidades_ufs:
                    quantidades_ufs[uf] = 1
                else:
                    quantidades_ufs[uf] = quantidades_ufs[uf] + 1
    return quantidades_ufs

In [12]:
ufs = quantidade_uf()

In [14]:
farmacias = farmacias[~farmacias.no_farmacia.isnull()]

In [16]:
farmacias = farmacias[~farmacias.uf.isnull()]

# Concentração de farmácias populares no territorio Brasileiro (mapa de calor)

In [18]:
locations = []

for index in range(0, len(farmacias)):
    lat = farmacias.loc[index]['lat']
    long = farmacias.loc[index]['long']
    locations.append([lat, long])

In [24]:
mapc = [-14.000, -50.000]
zoom = 4.4

htMap = folium.Map(location=mapc, zoom_start=zoom)
HeatMap(locations).add_to(htMap)
htMap

#### Análise do mapa de calor  
  
Cima encontra-se um mapa de calor que de formas simples pode nos levar a conclusões como:
  
  * A maior parte das farmacias populares estão nas regiões sul, suldeste e centro-oeste
  * A região nortedeste é menos provida de farmacias populares do que as outras três citadas acima
  * A região norte é estremamente carente de farmacias populares, o Amazonas possui quase todas a farmácias concentradas nos arredores de Manaus e o Amapá quase não possui esse tipo de estabelecimento  
  
Com isso é notavel que não existe uma uniformidade no numero de farmácias no território brasileiro.

# Farmacias por Estado brasileiro

In [26]:
# contagem com algoritimo linear de quantas farmacias por UF
quantidades_ufs = {}

for index in range(0, len(farmacias)):
    uf = farmacias.loc[index]['uf']
    if uf not in quantidades_ufs:
        quantidades_ufs[uf] = 1
    else:
        quantidades_ufs[uf] = quantidades_ufs[uf] + 1

In [22]:
abreviacao_ufs = []

for abreviacao in quantidades_ufs:
    abreviacao_ufs.append(abreviacao)
    
valores_ufs = []

for abreviacao in quantidades_ufs:
    valores_ufs.append(quantidades_ufs[abreviacao])

In [23]:
from bokeh.charts import Bar, show
from bokeh.models import HoverTool

data = {'uf': abreviacao_ufs, 'valores': valores_ufs}

p = Bar(data, label='uf', values='valores', width=1000, tools=None)
#p.toolbar.logo = None
p.toolbar_location = None

show(p)

E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='189bca89-9965-40fd-a88e-e5563a5c8d12', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='25f95b16-070f-42fd-9f34-79ac64148304', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='2bdab603-3a01-40e8-bb8a-937599e368a0', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='2c42a27e-21ed-4267-a83b-7d3dffba1d09', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='31fad99c-b04f-4a33-858e-681a8ec7b1b1', ...)
E-1010 (CDSVIEW

#### Análise do histograma  

Ao combinar os dados acima do histograma com dados como densidade demográfica e IDH dos estados temos:  
    
    * o IDH do Amapá é de 0,747 (o 12º mais elevado do Brasil) com uma população de 797.722, sendo que possui apenas 6 farmácias populares
    * o IDH de São Paulo é de 0,783 (o 2º mais elevado do Brasil) com uma população de 44.749.699, sendo que possui 4653 farmácias populares
    * o IDH de Minas Gerais é de 0,731 (o 9º mais elevado do Brasil) com uma população de 21.119.536, sendo que possui apenas 5155 farmácias populares
    
Com isso percebemos que mesmo com menos da metade do habitantes de São Paulo, Minas Gerais possui 502 farmacias populares a mais. E o Amapá possui possui apenas 6 estabelecimentos sendo assim 1 para cada 132.953 habitantes, em SP é 1 para cada 9617 habitantes e em MG 1 para cada 4096 habitantes.

In [28]:
Estados_populacao = pd.read_csv('datasets/mediPy/Estados_populacao.csv')
Estados_populacao

,UF,Populacao,% população
0,SP,45 094 866,"21,70%"
1,MG,21 119 536,"10,20%"
2,RJ,16 718 956,"8,10%"
3,BA,15 344 447,"7,40%"
4,RS,11 322 895,"5,50%"
5,PR,11 320 892,"5,50%"
6,PE,9 473 266,"4,60%"
7,CE,9 020 460,"4,40%"
8,PA,8 366 628,"4,00%"
9,SC,7 001 161,"3,40%"


In [38]:
# data frame 
por_farmacia = {}

for index in range(0, len(Estados_populacao)):
    uf = Estados_populacao.loc[index]['UF']
    farmacias = quantidades_ufs[uf]
    por_farmacia[uf] = int((Estados_populacao.loc[index]['Populacao']).replace(' ','')) / farmacias

TypeError: list indices must be integers or slices, not str